In [0]:

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M"),
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2020-01-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2021-12-31", '%Y-%m-%d')

RETENTION_D_LIST = ["D7","D14","D30","D60","D90"]

DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)


for retention_d in RETENTION_D_LIST:
    for granularity in DATE_DICT:
        for date in DATE_DICT[granularity]:
                try:
                    s3path ="s3://aardvark-prod-pdx-mdm-to-int/usage_retention/batch=retention_batch_{retention_d}_routine/version=v1.0.0/range_type=MONTH/date={date}/".format(retention_d=retention_d, date=date)
                    spark.read.parquet(s3path).createOrReplaceTempView("test_retention_day")
                    df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type, sum(value) as sum from test_retention_day where value is not null".format(date=date, granularity=granularity))
                    df.show(100)
                    # df = spark.sql("select distinct(device_type) from test_retention_day")
                    # df.show(100)
                    # df = spark.sql("select * from test_retention_week limit 20")
                    # print(df)
                    df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/2021-12-13/{}/".format(retention_d), mode="append") #append
                    print("PASS on {} {}".format(granularity, date))
                except Exception as e:
                    print("ERROR on {} {}".format(granularity, date))


In [0]:



spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/usage_retention/batch=retention_batch_D7_routine/version=v1.0.0/range_type=MONTH/date=2020-10-31").createOrReplaceTempView("tom_test7")
spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/usage_retention/batch=retention_batch_D14_routine/version=v1.0.0/range_type=MONTH/date=2020-10-31").createOrReplaceTempView("tom_test14")
spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/usage_retention/batch=retention_batch_D30_routine/version=v1.0.0/range_type=MONTH/date=2020-10-31").createOrReplaceTempView("tom_test30")
spark.sql("""select count(distinct(app_id)) from (
select distinct(app_id) from tom_test7 
union all 
select distinct(app_id) from tom_test14
union all 
select distinct(app_id) from tom_test30
)
""").show()

spark.sql("select * from tom_test7 limit 10 ").show()

In [0]:
%%sh

aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/usage_retention/batch=retention_batch_D7_routine/version=v1.0.0/range_type=MONTH/date=2021-10-31/


#batch=retention_batch_D7_routine/version=v1.0.0/range_type=MONTH/





In [0]:

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M"),
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2020-01-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2021-12-31", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)



for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
            try:
                s3path ="s3://aardvark-prod-pdx-mdm-to-int/usage_long_term_retention/version=v1.0.0/range_type=MONTH/date={date}/".format(retention_d=retention_d, date=date)
                spark.read.parquet(s3path).createOrReplaceTempView("test_retention_w_m")
                df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type, sum(value) as sum from test_retention_w_m where value is not null".format(date=date, granularity=granularity))
                df.show(100)
                # df = spark.sql("select distinct(device_type) from test_retention_day")
                # df.show(100)
                # df = spark.sql("select * from test_retention_week limit 20")
                # print(df)
                df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/2021-12-13/retention_W_M/", mode="append") #append
                print("PASS on {} {}".format(granularity, date))
            except Exception as e:
                print("ERROR on {} {}".format(granularity, date))


In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/2021-12-13/"


In [0]:



date='2021-12-13'

df = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/{}/D*/".format(date)).createOrReplaceTempView("tom_test")
spark.sql("select date,count_app_id,count_country, sum(sum) from tom_test group by date,count_app_id,count_country order by date asc").show(1000)
# df = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/{}/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/adhoc/retention_day.csv".format(date),header = 'true')



In [0]:
%%sh
DATE='2021-12-13'

rm -rf /tmp/$DATE/
mkdir -p /tmp/$DATE/

aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/adhoc/$DATE/ /tmp/$DATE/retention_day/
ls -al /tmp/$DATE/retention_day/

cat /tmp/$DATE/retention_day/*.csv

